In [1]:
import os
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1 import keras
from tensorflow.compat.v1.keras import layers,datasets,utils,models
from tensorflow.compat.v1.keras.preprocessing.image import ImageDataGenerator
from tensorflow.compat.v1.keras.models import Sequential, Model
from tensorflow.compat.v1.keras.layers import Dense, Dropout, Activation, Flatten,Input
from tensorflow.compat.v1.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalMaxPooling2D
import numpy as np
from tensorflow.compat.v1.keras.utils import to_categorical
from tensorflow.compat.v1.keras.regularizers import l2

from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

In [3]:
global_parameters = {'batch_size': 32,
                    'num_classes': 10,
                    'epochs': 30,
                     'leraning_rate': 0.0001,
                    'data_augmentation': True,
                     'user_dropout': True,
                     'dropout_rate': 0.1,
                     'user_l2': False,
                    'l2_lambda': 0.0001}

In [4]:
dirname = './data/cifar-10/test/'
testfile = []
for each in range(1, 300001):
    fname = str(each)+'.png'
    testfile.append(fname)
imglist = []

for each in testfile:
    with open(dirname+each,'rb') as fp:
        img = Image.open(fp)
    img = Image.open(dirname+each)
    img = np.array(img)
    imglist.append(img)

x_test = np.array(imglist)
x_test = x_test.astype('float32')
x_test /= 255.0

In [5]:
def conv_model():
    regularizer = None
    if global_parameters['user_l2']:
        regularizer = l2(global_parameters['l2_lambda'])
    inp = Input((32, 32, 3))
    x = Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizer)(inp)
    x = Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizer)(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizer)(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizer)(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizer)(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizer)(x)
    x = MaxPooling2D()(x)
    
    x = Flatten()(x)
    if global_parameters['user_dropout']:
        x = Dropout(0.10)(x)
    out = Dense(10, activation='softmax')(x)
    return Model(inputs=inp, outputs=out)

In [6]:
model = conv_model()
# opt = keras.optimizers.Adam(lr=global_parameters['leraning_rate'])
opt = keras.optimizers.SGD(lr=global_parameters['leraning_rate'])
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [7]:
model.load_weights('./model/impovementVGG16_weigths.h5')
pre_value = model.predict(x_test)

In [8]:
labels = np.argmax(pre_value, axis=1)
maps = {0:"airplane" ,1:"automobile" ,2:"bird ",3:"cat",4:"deer",5:"dog",6:"frog",7:"horse" ,8:"ship",9:"truck"}
maps_label = []
for each in labels:
    maps_label.append(maps[each])
maps_label = np.array(maps_label)

ids = np.array(range(1,labels.shape[0]+1))

In [9]:
temp = []
temp.append(ids)
temp.append(maps_label)
data = np.array(temp)

In [10]:
tdata = data.T
df = pd.DataFrame(tdata)
df.columns = ['id','label']
df.to_csv('./data/pd_data.csv')